In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:

import os
import numpy as np
import cv2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [ ]:
# preprocess.py

DATASET_DIR = "/content/drive/MyDrive/dataset"  # Update with your dataset path
IMG_SIZE = 224
SEQUENCE_LENGTH = 30

def extract_frames(video_path, sequence_length=SEQUENCE_LENGTH):
    frames = []
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = max(total_frames // sequence_length, 1)

    for i in range(sequence_length):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * frame_interval)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        frames.append(frame)

    while len(frames) < sequence_length:
        frames.append(frames[-1])

    cap.release()
    return np.array(frames)

def load_data():
    X, y = [], []

    for label, subdir in enumerate(["violence", "non_violence"]):
        subdir_path = os.path.join(DATASET_DIR, subdir)
        for video_file in os.listdir(subdir_path):
            video_path = os.path.join(subdir_path, video_file)
            frames = extract_frames(video_path)
            X.append(frames)
            y.append(label)

    X = np.array(X)
    y = np.array(y)
    return X, y

# Load and preprocess the data
X, y = load_data()
X = X / 255.0  # Normalize frames

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Save preprocessed data
np.save('/content/X_train.npy', X_train)
np.save('/content/X_test.npy', X_test)
np.save('/content/y_train.npy', y_train)
np.save('/content/y_test.npy', y_test)

print("Data preprocessing completed. Data saved as .npy files.")


In [ ]:
# feature_extraction.py

IMG_SIZE = 224
SEQUENCE_LENGTH = 30

# Load the pre-trained ResNet50 model for feature extraction
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
model = Model(inputs=base_model.input, outputs=GlobalAveragePooling2D()(base_model.output))

def extract_features(X):
    num_samples = X.shape[0]
    features = np.zeros((num_samples, SEQUENCE_LENGTH, model.output_shape[-1]))

    for i in range(num_samples):
        for j in range(SEQUENCE_LENGTH):
            features[i, j, :] = model.predict(np.expand_dims(X[i, j], axis=0))[0]

    return features

# Load preprocessed data
X_train = np.load('/content/X_train.npy')
X_test = np.load('/content/X_test.npy')

# Extract features
X_train_features = extract_features(X_train)
X_test_features = extract_features(X_test)

# Save extracted features
np.save('/content/X_train_features.npy', X_train_features)
np.save('/content/X_test_features.npy', X_test_features)

print("Feature extraction completed. Features saved as .npy files.")


In [ ]:
# model.py

def build_model(input_shape):
    model = tf.keras.Sequential([
        LSTM(64, input_shape=input_shape, return_sequences=True),
        Dropout(0.5),
        LSTM(32, return_sequences=False),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Define model input shape
input_shape = (30, 2048)  # Example input shape
model = build_model(input_shape)
model.summary()


In [ ]:
# train.py

# Load the extracted features and labels
X_train_features = np.load('/content/X_train_features.npy')
X_test_features = np.load('/content/X_test_features.npy')
y_train = np.load('/content/y_train.npy')
y_test = np.load('/content/y_test.npy')

# Build the model
input_shape = (X_train_features.shape[1], X_train_features.shape[2])
model = build_model(input_shape)

# Define model checkpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint('/content/violence_detection_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

# Train the model
history = model.fit(X_train_features, y_train, epochs=10, batch_size=16, validation_data=(X_test_features, y_test), callbacks=[checkpoint])

print("Training completed.")


In [ ]:
# evaluate.py

# Load the test features and labels
X_test_features = np.load('/content/X_test_features.npy')
y_test = np.load('/content/y_test.npy')

# Load the trained model
model = tf.keras.models.load_model('/content/violence_detection_model.h5')

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_features, y_test, verbose=1)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Predict on test set
y_pred = model.predict(X_test_features)
y_pred = (y_pred > 0.5).astype(int)

# Calculate accuracy manually
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")
